In [1]:
#!/usr/bin/env python3
import sys
import smach
import rospy
import cv2 as cv
import numpy as np
from std_srvs.srv import Empty
import moveit_commander
import moveit_msgs.msg
import geometry_msgs.msg
import tf2_ros as tf2
from tf2_sensor_msgs.tf2_sensor_msgs import do_transform_cloud
from utils_takeshi import *
from grasp_utils import *

In [2]:
    global head, wb, arm, tf_man, gaze, robot, scene #wbw, wbl
    global rgbd, hand_cam, wrist, gripper, grasp_base, clear_octo_client, service_client, AR_starter, AR_stopper

    moveit_commander.roscpp_initialize(sys.argv)
    rospy.init_node('takeshi_smach_20')

    head = moveit_commander.MoveGroupCommander('head')
    wb = moveit_commander.MoveGroupCommander('whole_body')
    arm =  moveit_commander.MoveGroupCommander('arm')
    # wbl = moveit_commander.MoveGroupCommander('whole_body_light')
    #wbw.set_workspace([-6.0, -6.0, 6.0, 6.0]) 
    #wbl.set_workspace([-6.0, -6.0, 6.0, 6.0])  
    wb.set_workspace([-6.0, -6.0, 6.0, 6.0])  
    
    robot = moveit_commander.RobotCommander()
    scene = moveit_commander.PlanningSceneInterface()
    
    tf_man = TF_MANAGER()
    rgbd = RGBD()
    hand_cam = HAND_RGB()
    wrist = WRIST_SENSOR()
    gripper = GRIPPER()
    grasp_base = OMNIBASE()
    gaze = GAZE()

    clear_octo_client = rospy.ServiceProxy('/clear_octomap', Empty)
    AR_starter = rospy.ServiceProxy('/marker/start_recognition',Empty)
    AR_stopper = rospy.ServiceProxy('/marker/stop_recognition',Empty)
    
    head.set_planning_time(0.3)
    head.set_num_planning_attempts(1)

[ WARN] [1668556356.810674974, 2510.938000000]: Link hand_l_finger_vacuum_frame has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1668556356.815118981, 2510.938000000]: Link head_l_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1668556356.815171882, 2510.938000000]: Link head_r_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1668556356.819554990, 2510.938000000]: Group state 'neutral' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.
[ WARN] [1668556356.819683791, 2510.938000000]: Group state 'go' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.

In [3]:
        # State initial
        try:
            clear_octo_client()
        except:
            print('cant clear octomap')
        AR_stopper.call()
        #Takeshi neutral
        arm.set_named_target('go')
        arm.go()
        gripper.steady()
        head.set_named_target('neutral')
        succ = head.go()
#         if succ:
#             return 'succ'
#         else:
#             return 'failed'

In [5]:
        #State look for AR marker
        try:
            AR_starter.call()
            clear_octo_client()
        except:
            print('cant clear octomap')
        rospy.sleep(0.15)
        arm.set_named_target('go')
        arm.go()
        acp = arm.get_current_joint_values()
        acp[0] += 0.15
        arm.set_joint_value_target(acp)
        arm.go()
        hcp = gaze.relative(1,0,0.6)
        head.set_joint_value_target(hcp)
        head.go()
        succ = False
        while not succ:
            grasp_base.tiny_move(velT=0.3, std_time=0.3)
            trans,rot = tf_man.getTF(target_frame='ar_marker/201', ref_frame='base_link')
            print(trans)
            succ = type(trans) is not bool
            if succ:
                tf_man.pub_static_tf(pos=trans, rot=rot, point_name='cassette', ref='base_link')
                tf_man.change_ref_frame_tf(point_name='cassette', new_frame='map')
                arm.set_named_target('go')
                arm.go()
        

[0.5594506360855734, 0.038678126954500944, 0.7706026943592809]


In [4]:
        try:
            AR_starter.call()
            clear_octo_client()
        except:
            print('cant clear octomap')
        rospy.sleep(0.2)
        arm.set_named_target('go')
        arm.go()
        hcp = gaze.relative(1,0,0.7)
        head.set_joint_value_target(hcp)
        head.go()
        succ = False
        flag = 1
        while not succ:
            trans,rot = tf_man.getTF(target_frame='ar_marker/201', ref_frame='base_link')
            print(trans)
            succ = type(trans) is not bool
            if succ:
                tf_man.pub_static_tf(pos=trans, rot=rot, point_name='cassette', ref='base_link')
                rospy.sleep(0.5)
                while not tf_man.change_ref_frame_tf(point_name='cassette', new_frame='map'):
                    print('change reference frame is not done yet')
                arm.set_named_target('go')
                arm.go()
#                 return 'succ'
            else:
                if flag == 1:
                    hcp = gaze.relative(0.7,0.5,0.7)
                    head.set_joint_value_target(hcp)
                    head.go()
                    flag += 1
                    rospy.sleep(0.3)
                elif flag == 2:
                    hcp = gaze.relative(0.7,-0.5,0.7)
                    head.set_joint_value_target(hcp)
                    head.go()
                    flag += 1
                    rospy.sleep(0.3)
                else:
                    head.set_named_target('neutral')
                    head.go() 
#                     return 'failed'

[0.6311049463634851, 0.104216237402557, 0.7697673407304418]


In [6]:
        # State AR alignment
        arm.set_named_target('go')
        arm.go()
        succ = False
        THRESHOLD = 0.08
        #talk("I am going to align with the table")
        while not succ:
            try:
                trans, rot = tf_man.getTF(target_frame='cassette', ref_frame='base_link')
                euler = tf.transformations.euler_from_quaternion(rot)
                theta = euler[2]
                e = theta + 1.57
                print(e)
                if abs(e) < THRESHOLD:
#                     talk("ready")
                    succ = True
                else:
                    rospy.sleep(0.1)
                    grasp_base.tiny_move(velT = 0.5*e, std_time=0.1)
            except:
                hcp[0] -= 0.2
                if hcp[0] > -1.2:
                    hcp[0] = 0.0
                head.set_joint_value_target(hcp)
                head.go()
                
        
        if succ:
            return 'succ'
        else:
            return 'failed'

-0.26248861545981206
-0.26223352277510825
-0.26223352277510825
-0.2623951318970188
-0.24832372777446787
-0.23624388092338489
-0.20520276763142586
-0.1483569929651225
-0.1483569929651225
-0.1483569929651225
-0.12105098677657988
-0.06961465322951454


SyntaxError: 'return' outside function (1018355891.py, line 29)

In [7]:
##Working good
arm.set_named_target('neutral')
arm.go()
trans,_ = tf_man.getTF(target_frame='cassette', ref_frame='odom')
pos, rot =tf_man.getTF(target_frame='hand_palm_link', ref_frame='odom')
pose_goal = geometry_msgs.msg.Pose()
pose_goal.orientation.w = rot[3]
pose_goal.orientation.x = rot[0]
pose_goal.orientation.y = rot[1]
pose_goal.orientation.z = rot[2]
pose_goal.position.x = pos[0]
pose_goal.position.y = pos[1]
pose_goal.position.z = trans[2] + 0.1
arm.set_start_state_to_current_state()
arm.set_pose_target(pose_goal)
result = arm.go()
print(result)
succ = False
THRESHOLD = 0.01
while not succ:
    trans,_ = tf_man.getTF(target_frame='cassette', ref_frame='hand_palm_link')
    print(trans)
    if abs(trans[1])>THRESHOLD:
        grasp_base.tiny_move(velY=-trans[1], std_time=0.2)
    else:
        succ = True
succ = False
while not succ:
    trans,_ = tf_man.getTF(target_frame='cassette', ref_frame='hand_palm_link')
    if abs(trans[2])>0.1:
        grasp_base.tiny_move(velX=trans[2], std_time=0.2)
    else:
        succ = True

True
[-0.09516926955922167, 0.038542194642356545, 0.2858203458833154]
[-0.09950995562160692, 0.037895559870616255, 0.2841664899488054]
[-0.10015268012386158, 0.03191685866435631, 0.2848441185333288]
[-0.10015174294043894, 0.026340036401830497, 0.28478016611679524]
[-0.1001571606931746, 0.021288066654262444, 0.2841341033581774]
[-0.10016021328562641, 0.019727722621463634, 0.2860212342579942]
[-0.10016446491430053, 0.01457568398682528, 0.2860163348208422]
[-0.10032706700324379, 0.01200423269563397, 0.28617817690900205]
[-0.10024591683934614, 0.006758793700859134, 0.28692209657971196]


In [4]:
succ = False
THRESHOLD = 0.01
while not succ:
    trans,_ = tf_man.getTF(target_frame='cassette', ref_frame='hand_palm_link')
    print(trans)
    if abs(trans[1])>THRESHOLD:
        grasp_base.tiny_move(velY=-0.2*trans[1], std_time=0.2)
    else:
        succ = True
succ = False
while not succ:
    trans,_ = tf_man.getTF(target_frame='cassette', ref_frame='hand_palm_link')
    if abs(trans[2])>0.1:
        grasp_base.tiny_move(velX=0.2*trans[2], std_time=0.2)
    else:
        succ = True

False


TypeError: 'bool' object is not subscriptable

In [36]:
# print(trans)
gripper.open()
# trans,rot= tf_man.getTF(target_frame='cassette', ref_frame='odom')
rospy.sleep(0.5)
tf_man.pub_static_tf(pos=[0,0,0.07] ,point_name='grasp1', ref='hand_palm_link')
trans, rot = tf_man.getTF(target_frame='grasp1', ref_frame='odom')
# print(trans, trans1)
pose_goal = geometry_msgs.msg.Pose()
pose_goal.orientation.w = rot[3]
pose_goal.orientation.x = rot[0]
pose_goal.orientation.y = rot[1]
pose_goal.orientation.z = rot[2]
# pose_goal.orientation.
pose_goal.position.x = trans[0]
pose_goal.position.y = trans[1]
pose_goal.position.z = trans[2]

wbw.set_start_state_to_current_state()
wbw.set_pose_target(pose_goal)
succ, plan=wbw.plan()

(True,
 joint_trajectory: 
   header: 
     seq: 0
     stamp: 
       secs: 0
       nsecs:         0
     frame_id: "odom"
   joint_names: 
     - arm_lift_joint
     - arm_flex_joint
     - arm_roll_joint
     - wrist_flex_joint
     - wrist_roll_joint
     - wrist_ft_sensor_frame_joint
   points: 
     - 
       positions: [3.649947447585504e-05, -0.00030452801888536385, -1.569931345343694, -1.5700146302849658, 0.00046110822381617567, 0.0]
       velocities: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
       accelerations: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
       effort: []
       time_from_start: 
         secs: 0
         nsecs:         0
     - 
       positions: [0.008143937286211675, -0.146913015685917, -1.5312160081384125, -1.5578494393099107, 0.14698458733819678, 5e-324]
       velocities: [0.0030758951415370012, -0.05562205290928529, 0.014688280117340295, 0.004615373276360734, 0.05558980136444953, 0.0]
       accelerations: [3.290699007239642e-19, 0.0, 0.0, 3.1919780370224526e-17, 0.0, 0.0

In [27]:
        gripper.open()
        acp = arm.get_current_joint_values()
        acp[0] -= 0.03
        arm.set_joint_value_target(acp)
        arm.go()

True

In [28]:
        #         gripper.open()
        gripper.close()
        rospy.sleep(0.5)
        acp = arm.get_current_joint_values()
        acp[0]+=0.03
        arm.set_joint_value_target(acp)
        arm.go()
        rospy.sleep(0.3)
        
        force = wrist.get_force()
        print(force)
        if abs(force[0]) > 1:
#             talk('i have the cassette')
#             return 'succ'
            print('si')
        else:
            gripper.open()
            acp[0]-=0.03
            arm.set_joint_value_target(acp)
            arm.go()
#             talk('i will try again')
#             return 'tries'

[2.1469955063325665, -0.5547853281515758, 2.9752560460507467]
si


In [29]:
scene = moveit_commander.PlanningSceneInterface()
##Adding objects to planning scene
box_pose = geometry_msgs.msg.PoseStamped()
box_pose.header.frame_id = "hand_palm_link"
box_pose.pose.orientation.w = 1.0
box_pose.pose.position.z =  0.08  # below the panda_hand frame
box_name = "box"
scene.add_box(box_name, box_pose, size=(0.1, 0.02, 0.1))

In [31]:
##attaching object to the gripper

robot = moveit_commander.RobotCommander()
eef_link = arm.get_end_effector_link()
grasping_group = "gripper"
touch_links = robot.get_link_names(group=grasping_group)
scene.attach_box(eef_link, box_name, touch_links=touch_links)

In [19]:
##Detaching objects
eef_link = arm.get_end_effector_link()
scene.remove_attached_object(eef_link, name=box_name)

##Remove objects
scene.remove_world_object(box_name)

In [11]:
        FI = wrist.get_force()
#         gripper.close()
#         gripper.close()
        succ = False
#         trans, rot = tf_man.getTF(target_frame='hand_palm_link', ref_frame='odom') 
        # tf_man.pub_static_tf(point_name=goal, pos)
#         trans[2] += 0.03
        #         while not succ:
#         pose_goal = set_pose_goal(pos = trans, rot =rot)
            # joint_goal = wb.get_current_joint_values()
            # if len(joint_goal)>0:
                # joint_goal[0] += 0.03
#         arm.set_pose_target(pose_goal)
        joint_goal = arm.get_current_joint_values()
        joint_goal[0] +=0.03
        arm.set_joint_value_target(joint_goal)
        succ = arm.go()
        rospy.sleep(1.0)
        # rospy.sleep(1.0)
        FF = wrist.get_force()
        if abs(FF[0]-FI[0]) > 1.5:
            rospy.sleep(0.5)
            talk('i have the cassette')
            return 'succ'
            # print('si')
        else:
            gripper.open()
            rospy.sleep(0.5)
            # acp[0]-= 0.03
            trans[2] -= 0.03
            pose_goal = set_pose_goal(pos = trans, rot=rot)
            arm.set_pose_target(acp)
            arm.go()
            talk('i will try again')
            return 'tries'

SyntaxError: 'return' outside function (2669349467.py, line 24)

shutdown request: [/Pruebas_de_graspeo_v2] Reason: new node registered with same name


In [8]:
gripper.open()

In [ ]:
        # State Pre grasp pose
        succ = False
#         talk("I will reach the cassette")
        gripper.open()
        height = 0
        try:
#             t = tfbuff.lookup_transform('base_link','ar_marker/201',rospy.Time(0))
#             trans,_ = tf2_obj_2_arr(t)
            trans,_ =tf_man.getTF(target_frame='cassette', ref_frame='base_link')
            height = trans[2]
#         grasp_from_above_joints = [0.59,-1.3376,0,-1.8275,0.0,0.0]
        except:
            print('no pude bebe')
        print(height)
        grasp_from_above_joints = [height - 0.102,-1.3376,0,-1.8275,0.0,0.0]
        arm.set_joint_value_target(grasp_from_above_joints)
        succ = arm.go()
        if succ:
            return 'succ'
        else:
            return 'failed'

In [ ]:
        # State AR adjustment

        try:
            clear_octo_client()
        except:
            print('cant clear octomap')
        #Takeshi gets close to the cassette
        AR_starter.call()
        succ = False
        THRESHOLD = 0.025
        succ = False
        while not succ:
            try:
                trans,rot = tf_man.getTF(target_frame='cassette', ref_frame='hand_palm_link')
                ex=trans[0]
                ey=-trans[1]
                print(ex, ey)
                if abs(ex) > THRESHOLD:
                    grasp_base.tiny_move(velX = ex, MAX_VEL = 0.05)#, y = -traf.y + Y_OFFSET)
                if abs(ey) > THRESHOLD:
                    grasp_base.tiny_move(velY = ey, MAX_VEL = 0.05)
                if (abs(ex) <= THRESHOLD and abs(ey) <= THRESHOLD):
                    hcp[0] = 0
                    head.set_joint_value_target(hcp)
                    head.go()
                    talk("I am almost there")
                    succ = True
            except:
                print('lost')
                break
        if succ:
            stopper.call()
            return 'succ'
        else:
            return 'failed'

In [6]:
arm.set_named_target('neutral')
arm.go()
# AR_starter.call()
# whole_body.set_num_planning_attempts(100)
# whole_body.set_planning_time(2)
gripper.open()

In [ ]:
# import geometry_msgs.msg


In [ ]:
# scene.remove_world_object(box_name)
arm.set_named_target('neutral')
arm.go()

In [ ]:
# tf_man.getTF(target_frame=)
# scene = moveit_commander.PlanningSceneInterface()
##Adding objects to planning scene
# box_pose = geometry_msgs.msg.PoseStamped()
# box_pose.header.frame_id = "hand_palm_link"
# box_pose.pose.orientation.w = 1.0
# box_pose.pose.position.z =  0.11
# box_name = "box"
# scene.add_box(box_name, box_pose, size=(0.1, 0.02, 0.1))


tf_man.pub_static_tf(point_name='grasp1', pos=[0,0,0.02], ref='hand_palm_link')
# rospy.sleep(0.5)?
trans, rot = tf_man.getTF(target_frame='grasp1', ref_frame='odom')
print(trans,rot)
# rospy.sleep(0.5)
pose_goal = geometry_msgs.msg.Pose()
# pose_goal.orientation.w = 1
# pose_goal.orientation.x = rot[0]
# pose_goal.orientation.y = rot[1]
# pose_goal.orientation.z = rot[2]
# pose_goal.orientation.
pose_goal.position.x = trans[0]
pose_goal.position.y = trans[1]
pose_goal.position.z = trans[2]

whole_body.set_start_state_to_current_state()
# arm.set_pose_target(pose_goal)
whole_body.set_position_target(trans)
whole_body.go()

In [ ]:
        # State color adjustment
        try:
            clear_octo_client()
        except:
            print('cant clear octomap')

        #Takeshi detects the cassette by color and go for it
        succ = False
        THRESHOLD = 15
        goalPos = [258.61,261.75]
        while not succ:
            [currentPos] = hand_cam.color_segmentator(color = 'orange')
#     print(currentPos)
            ex = -(goalPos[0]-currentPos[0]) 
            ey = (goalPos[1]-currentPos[1])
            print(ex, ey)
            if abs(ex) > THRESHOLD:
                grasp_base.tiny_move(velX = ex, std_time=0.1, MAX_VEL=0.01)#, y = -traf.y + Y_OFFSET)
                rospy.sleep(0.5)
            if abs(ey) > THRESHOLD:
                grasp_base.tiny_move(velY = ey, std_time=0.1, MAX_VEL=0.01)
                rospy.sleep(0.5)
            if (abs(ex) <= THRESHOLD and abs(ey) <= THRESHOLD):
#                 talk("done, now i will take it")
                succ = True
        if succ:
            return 'succ'
        else:
            return 'failed'

In [ ]:
        # State grasp table
        gripper.open()
        rospy.sleep(0.3)
        acp = arm.get_current_joint_values()
        acp[0] -= 0.03
#         acp = [0.56,-1.3376,0,-1.8275,0.0,0.0]
        arm.set_joint_value_target(acp)
        arm.go()
        gripper.close()
        rospy.sleep(0.3)
        
        check_grasp_joints=[0.69,-1.3376,0,-1.8275,0.0,0.0]
        arm.set_joint_value_target(check_grasp_joints)
        arm.go()
        
        check_grasp_joints=[0.69,-1.3376,-0.8,-1.8275,0.0,0.0]
        arm.set_joint_value_target(check_grasp_joints)
        arm.go()

        THRESHOLD = 30
        goalPos = [233.80,268.74]
        [currentPos] = hand_cam.color_segmentator(color = 'orange')
        ex = -(goalPos[0]-currentPos[0]) 
        ey = (goalPos[1]-currentPos[1])
        if (abs(ex) <= THRESHOLD and abs(ey) <= THRESHOLD):
            talk("I have the cassete")
            return 'succ'
        else:
            talk("Something went wrong, i will try again")
            return 'tries'

In [ ]:
gripper.open()

In [ ]:
        # State post grasp pose
        acp = arm.get_current_joint_values()
        acp[0] = 0.69
        arm.set_joint_value_target(acp)
        arm.go()
        rospy.sleep(0.3)
        grasp_base.tiny_move(velX = -0.8, std_time=0.8, MAX_VEL=0.08)
        arm.set_named_target('go')
        succ = arm.go()
        if succ:
            return 'succ'
        else:
            return 'failed'

In [ ]:
# State right shift
        grasp_base.tiny_move(velY = -0.8, std_time=0.8)
        succ = True
        if succ:
            return 'succ'
        else:
            return 'failed'

In [ ]:
gripper.steady()

In [ ]:
# rospy.init_node('Pruebas_TF')
# tf_man = TF_MANAGER()
rospy.sleep(0.5)
tf_man.pub_static_tf(trans = [1,0,1], pointName = 'marker', ref='head_rgbd_sensor_link')

In [ ]:
tf_man.change_ref_frame_tf(tf_name='marker', newFrame='map')

In [ ]:
gripper = GRIPPER()
gripper.steady()

In [ ]:
trans, rot = tf_man.getTF('base_link', 'marker')
# gaze = GAZE()
print(trans)
gaze.relative(trans[0],trans[1],trans[2])

In [13]:
# tf_man.pub_static_tf(point_name='cassette', pos=[0.5,0,0.8 ], ref='base_link')
tf_man.change_ref_frame_tf(point_name='cassette', new_frame='map')

True

In [9]:
        #PRE GRASP POSE
        gripper.open()
        arm.set_named_target('neutral')
        ready = arm.go()
        result = False
        if ready :
            trans,_ = tf_man.getTF(target_frame='cassette', ref_frame='hand_palm_link')
            pos,_ = tf_man.getTF(target_frame='hand_palm_link', ref_frame='odom')
            while (type(trans) is bool) and (type(pos) is bool) :
                trans,_ = tf_man.getTF(target_frame='cassette', ref_frame='hand_palm_link')
                pos,_ = tf_man.getTF(target_frame='hand_palm_link', ref_frame='odom')
            print(trans[2])
            tf_man.pub_tf(point_name='goal_pose', pos=[trans[2]+0.1, 0.0, 0.11], ref='hand_palm_link')
            rospy.sleep(0.5)
            pos, rot = tf_man.getTF(target_frame='goal_pose', ref_frame='odom')
            if type(pos) is not bool:
                pose_goal = set_pose_goal(pos=pos, rot=rot)
                arm.set_start_state_to_current_state() 
                arm.set_pose_target(pose_goal)
                succ, plan, _, _ = arm.plan()
                if succ:
                    arm.execute(plan)
                else:
                    rospy.loginfo('i could not plan to goal pose')

[ WARN] [1668557634.261649675, 2834.197000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2834.230000 according to authority unknown_publisher
[ WARN] [1668557634.261717774, 2834.197000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2834.230000 according to authority /pose_integrator
[ WARN] [1668557634.261774266, 2834.197000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2834.230000 according to authority /pose_integrator


0.21335275218483313


[ WARN] [1668557643.018518922, 2836.397000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2836.435000 according to authority /pose_integrator
[ WARN] [1668557643.018731617, 2836.397000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2836.435000 according to authority /pose_integrator
[ WARN] [1668557643.019201167, 2836.397000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2836.435000 according to authority unknown_publisher
[ WARN] [1668557644.980227563, 2836.884000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2836.937000 according to authority unknown_publisher
[ WARN] [1668557644.981756839, 2836.884000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2836.937000 according to authority /pose_integrator
[ WARN] [1668557644.981834334, 2836.884000000]: TF_REPEATED_DATA ignoring data with redundant timestamp

In [14]:
        succ = False
        THRESHOLD = 0.01
        
        while not succ:
            trans,_ = tf_man.getTF(target_frame='cassette', ref_frame='hand_palm_link')
            if type(trans) is not bool:
                _, eY, eX = trans
                rospy.loginfo("Distance to goal: {:.2f}, {:.2f}".format(eX, eY))
                if abs(eY) < THRESHOLD:
                    eY = 0
                if abs(eX) < THRESHOLD:
                    eX = 0
                succ =  eX == 0 and eY == 0
                    # grasp_base.tiny_move(velY=-0.4*trans[1], std_time=0.2, MAX_VEL=0.3)
                grasp_base.tiny_move(velX=0.3*eX, velY=-0.4*eY, std_time=0.2, MAX_VEL=0.3) #Pending test

[INFO] [1668560041.785698, 3424.127000]: Distance to goal: 0.10, 0.08
[INFO] [1668560043.223311, 3424.331000]: Distance to goal: 0.10, 0.08
[INFO] [1668560044.064017, 3424.597000]: Distance to goal: 0.09, 0.07
[INFO] [1668560045.282004, 3424.806000]: Distance to goal: 0.08, 0.06
[INFO] [1668560046.325185, 3425.023000]: Distance to goal: 0.08, 0.06
[INFO] [1668560047.109939, 3425.247000]: Distance to goal: 0.07, 0.05
[INFO] [1668560048.047769, 3425.450000]: Distance to goal: 0.07, 0.05
[INFO] [1668560049.312761, 3425.650000]: Distance to goal: 0.07, 0.05
[INFO] [1668560050.366034, 3425.984000]: Distance to goal: 0.06, 0.04
[INFO] [1668560051.564851, 3426.184000]: Distance to goal: 0.05, 0.04
[INFO] [1668560052.783723, 3426.389000]: Distance to goal: 0.05, 0.03
[INFO] [1668560053.587619, 3426.593000]: Distance to goal: 0.05, 0.03
[INFO] [1668560054.722661, 3426.800000]: Distance to goal: 0.05, 0.03
[INFO] [1668560055.879425, 3427.024000]: Distance to goal: 0.04, 0.03
[INFO] [1668560056.7

[ WARN] [1668560066.746323566, 3429.183000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3429.234000 according to authority /pose_integrator
[ WARN] [1668560066.746408955, 3429.183000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3429.234000 according to authority /pose_integrator
[ WARN] [1668560066.749832134, 3429.183000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3429.234000 according to authority unknown_publisher


[INFO] [1668560067.516910, 3429.368000]: Distance to goal: 0.02, 0.01
[INFO] [1668560068.858840, 3429.569000]: Distance to goal: 0.02, 0.01
[INFO] [1668560069.808597, 3429.806000]: Distance to goal: 0.02, 0.01


[ WARN] [1668560069.917225596, 3429.806000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3429.834000 according to authority /pose_integrator
[ WARN] [1668560069.917315665, 3429.806000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3429.834000 according to authority /pose_integrator
[ WARN] [1668560069.919723291, 3429.806000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3429.834000 according to authority unknown_publisher


[INFO] [1668560070.551892, 3430.027000]: Distance to goal: 0.02, 0.01
[INFO] [1668560072.036791, 3430.228000]: Distance to goal: 0.02, 0.01
[INFO] [1668560073.246082, 3430.442000]: Distance to goal: 0.01, 0.01
[INFO] [1668560074.115214, 3430.661000]: Distance to goal: 0.01, 0.01
[INFO] [1668560075.283920, 3430.863000]: Distance to goal: 0.01, 0.01
[INFO] [1668560076.559885, 3431.063000]: Distance to goal: 0.01, 0.01
[INFO] [1668560077.532057, 3431.280000]: Distance to goal: 0.01, 0.01
[INFO] [1668560078.694908, 3431.496000]: Distance to goal: 0.01, 0.01


...

[ WARN] [1668561244.475181232, 3719.849000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3719.885000 according to authority /pose_integrator
[ WARN] [1668561244.475339973, 3719.849000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3719.885000 according to authority /pose_integrator
[ WARN] [1668561244.477266140, 3719.849000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3719.885000 according to authority unknown_publisher
[ WARN] [1668561250.574095990, 3721.496000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3721.531000 according to authority /pose_integrator
[ WARN] [1668561250.574175009, 3721.496000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3721.531000 according to authority /pose_integrator
[ WARN] [1668561250.574744558, 3721.496000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

[ WARN] [1668561416.802437265, 3761.717000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3761.749000 according to authority /pose_integrator
[ WARN] [1668561416.802529660, 3761.717000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3761.749000 according to authority /pose_integrator
[ WARN] [1668561416.804166346, 3761.717000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3761.749000 according to authority unknown_publisher
[ WARN] [1668561430.357244729, 3764.957000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3765.036000 according to authority unknown_publisher
[ WARN] [1668561430.365697559, 3765.036000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3765.036000 according to authority /pose_integrator
[ WARN] [1668561430.365781385, 3765.036000000]: TF_REPEATED_DATA ignoring data with redundant timestamp

[ WARN] [1668561615.235667486, 3813.323000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3813.354000 according to authority /pose_integrator
[ WARN] [1668561615.235766464, 3813.323000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3813.354000 according to authority /pose_integrator
[ WARN] [1668561615.237175711, 3813.323000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3813.354000 according to authority unknown_publisher
[ WARN] [1668561630.558221128, 3817.156000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3817.216000 according to authority /pose_integrator
[ WARN] [1668561630.558334294, 3817.156000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3817.216000 according to authority /pose_integrator
[ WARN] [1668561630.558720399, 3817.156000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

[ WARN] [1668561686.866930107, 3832.088000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3832.199000 according to authority unknown_publisher
[ WARN] [1668561686.868361767, 3832.088000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3832.199000 according to authority /pose_integrator
[ WARN] [1668561686.868433213, 3832.088000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3832.199000 according to authority /pose_integrator
[ WARN] [1668561688.972221566, 3832.816000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3832.870000 according to authority /pose_integrator
[ WARN] [1668561688.972332547, 3832.816000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3832.870000 according to authority /pose_integrator
[ WARN] [1668561688.972916428, 3832.816000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

[ WARN] [1668561805.256859259, 3864.363000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3864.395000 according to authority unknown_publisher
[ WARN] [1668561805.259091569, 3864.363000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3864.395000 according to authority /pose_integrator
[ WARN] [1668561805.259171651, 3864.363000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3864.395000 according to authority /pose_integrator
[ WARN] [1668561812.287368963, 3865.995000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3866.016000 according to authority unknown_publisher
[ WARN] [1668561812.289458682, 3866.016000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3866.016000 according to authority /pose_integrator
[ WARN] [1668561812.290420808, 3866.016000000]: TF_REPEATED_DATA ignoring data with redundant timestamp

In [20]:
        from geometry_msgs.msg import PoseStamped
        FI = wrist.get_force()
        rospy.sleep(1.0)
        eef_link = arm.get_end_effector_link()
        ##Adding objects to planning scene
        box_pose = PoseStamped()
        box_pose.header.frame_id = "hand_palm_link"
        box_pose.pose.orientation.w = 1.0
        box_pose.pose.position.z =  0.05  # below the panda_hand frame
        box_name = "box"
        scene.add_box(box_name, box_pose, size=(0.075, 0.05, 0.075))
        rospy.sleep(0.7)
        ##attaching object to the gripper
        grasping_group = "arm"
        touch_links = robot.get_link_names(group=grasping_group)
        scene.attach_box(eef_link, box_name, touch_links=touch_links)
        # gripper.close()
        gripper.close()
        succ = False
        trans, rot = tf_man.getTF(target_frame='hand_palm_link', ref_frame='odom') 
        trans[2] += 0.03
        # while not succ:
        pose_goal = set_pose_goal(pos = trans, rot =rot)
        arm.set_start_state_to_current_state()
        arm.set_pose_target(pose_goal)
        succ, plan, _, _ = arm.plan()
        if succ:
            arm.execute(plan)
        else:
            return 'tries'
        rospy.sleep(0.5)
        FF = wrist.get_force()
        if abs(FF[2]-FI[2]) > 1.5:
            rospy.sleep(0.5)
            talk('i have the cassette')
            return 'succ'
        else:
            gripper.open()
            rospy.sleep(0.5)
            # acp[0]-= 0.03
            trans[2] -= 0.03
            pose_goal = set_pose_goal(pos = trans, rot=rot)
            arm.set_start_state_to_current_state()
            arm.set_pose_target(pose_goal)
            succ, plan, _, _ = arm.plan()
            if succ:
                arm.execute(plan)
            talk('i will try again')
            return 'tries'

[ WARN] [1668562960.197934206, 4136.530000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4136.547000 according to authority unknown_publisher
[ WARN] [1668562960.199567301, 4136.530000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4136.547000 according to authority /pose_integrator
[ WARN] [1668562960.199633176, 4136.530000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4136.547000 according to authority /pose_integrator
[ WARN] [1668562971.612212094, 4139.232000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4139.289000 according to authority unknown_publisher
[ WARN] [1668562971.616073717, 4139.232000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4139.289000 according to authority /pose_integrator
[ WARN] [1668562971.616391994, 4139.232000000]: TF_REPEATED_DATA ignoring data with redundant timestamp

SyntaxError: 'return' outside function (203946699.py, line 30)

[ WARN] [1668562983.983223985, 4142.170000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4142.191000 according to authority unknown_publisher
[ WARN] [1668562983.986592788, 4142.170000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4142.191000 according to authority /pose_integrator
[ WARN] [1668562983.986659035, 4142.170000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4142.191000 according to authority /pose_integrator
[ WARN] [1668562985.981524894, 4142.597000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4142.625000 according to authority unknown_publisher
[ WARN] [1668562985.981962807, 4142.597000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4142.625000 according to authority /pose_integrator
[ WARN] [1668562985.982035000, 4142.597000000]: TF_REPEATED_DATA ignoring data with redundant timestamp

[ WARN] [1668563171.717271735, 4190.025000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4190.043000 according to authority /pose_integrator
[ WARN] [1668563171.717341338, 4190.025000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4190.043000 according to authority /pose_integrator
[ WARN] [1668563171.717952196, 4190.026000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4190.043000 according to authority unknown_publisher
[ WARN] [1668563172.797571825, 4190.273000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4190.304000 according to authority /pose_integrator
[ WARN] [1668563172.797654079, 4190.273000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4190.304000 according to authority /pose_integrator
[ WARN] [1668563172.801159706, 4190.273000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

[ WARN] [1668563332.143514423, 4230.848000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4230.869000 according to authority /pose_integrator
[ WARN] [1668563332.143602666, 4230.848000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4230.869000 according to authority /pose_integrator
[ WARN] [1668563332.143859905, 4230.848000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4230.869000 according to authority unknown_publisher
[ WARN] [1668563355.923546025, 4236.124000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4236.162000 according to authority /pose_integrator
[ WARN] [1668563355.923642170, 4236.124000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4236.162000 according to authority /pose_integrator
[ WARN] [1668563355.923828660, 4236.124000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

[ WARN] [1668563490.805017180, 4268.597000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4268.644000 according to authority unknown_publisher
[ WARN] [1668563490.805790394, 4268.597000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4268.644000 according to authority /pose_integrator
[ WARN] [1668563490.805876684, 4268.597000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4268.644000 according to authority /pose_integrator
[ WARN] [1668563497.634147340, 4270.340000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4270.380000 according to authority unknown_publisher
[ WARN] [1668563497.638030559, 4270.340000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4270.380000 according to authority /pose_integrator
[ WARN] [1668563497.638472475, 4270.340000000]: TF_REPEATED_DATA ignoring data with redundant timestamp

[ WARN] [1668563665.596529884, 4311.541000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4311.541000 according to authority unknown_publisher
[ WARN] [1668563665.597776734, 4311.541000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4311.541000 according to authority /pose_integrator
[ WARN] [1668563665.597897258, 4311.541000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4311.541000 according to authority /pose_integrator
[ WARN] [1668563684.008112793, 4316.152000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4316.152000 according to authority unknown_publisher
[ WARN] [1668563684.020883139, 4316.152000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4316.152000 according to authority /pose_integrator
[ WARN] [1668563684.020970051, 4316.152000000]: TF_REPEATED_DATA ignoring data with redundant timestamp

[ WARN] [1668563848.329620837, 4355.908000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4355.940000 according to authority unknown_publisher
[ WARN] [1668563848.332108284, 4355.908000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4355.940000 according to authority /pose_integrator
[ WARN] [1668563848.332177100, 4355.908000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4355.940000 according to authority /pose_integrator
